In [1]:
import pandas as pd
import numpy as np
import csv
from tqdm import tqdm
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import date
from os.path import join, isfile
from os import listdir
import time
from sklearn.metrics import f1_score
import tensorflow as tf 
from statistics import mode

In [2]:
import pointnet_model

In [3]:
def read_rep(syn_name, syn_csv, ID_csv, data_dir):
    
    # open directories
    syn_dir = data_dir+"\\{}-patients".format(syn_name)
    ID_dir = data_dir+ "\\{}-selected-ID-controls".format(syn_name)

    # get list of filenames
    files_syn = [f for f in listdir(syn_dir) if (isfile(join(syn_dir, f))) and ".jpg" in f]
    files_ID = [f for f in listdir(ID_dir) if (isfile(join(ID_dir, f))) and ".jpg" in f]
    
    data_syn, data_ID, labels_syn, labels_ID = [], [], [], []

    with open (syn_csv, newline='') as file:
        reader = csv.reader(file, delimiter=',')
        for index, row in enumerate(reader):
            if row[0] in files_syn: 
                rep = list(map(float, row[1:]))
                data_syn.append(rep)
                labels_syn.append(1)

    with open (ID_csv, newline='') as file:
        reader = csv.reader(file, delimiter=',')
        for index, row in enumerate(reader):
            if row[0] in files_ID:
                rep = list(map(float, row[1:]))
                data_ID.append(rep)
                labels_ID.append(0)

    return np.array(data_syn), np.array(data_ID), np.array(labels_syn), np.array(labels_ID)

In [4]:
def read_rep_landmarks(syn_name, syn_csv, ID_csv, data_dir):
    
    # open directories
    syn_dir = data_dir+"\\{}-patients".format(syn_name)
    ID_dir = data_dir+ "\\{}-selected-ID-controls".format(syn_name)

    # get list of filenames
    files_syn = [f for f in listdir(syn_dir) if (isfile(join(syn_dir, f))) and ".jpg" in f]
    files_ID = [f for f in listdir(ID_dir) if (isfile(join(ID_dir, f))) and ".jpg" in f]

    data_syn, data_ID, labels_syn, labels_ID = [], [], [], []

    with open (syn_csv, newline='') as file:
        reader = csv.reader(file, delimiter=',')
        for index, row in enumerate(reader):
            if row[0] in files_syn:
                rep = []
                i = 1
                while i < len(row[1:]):
                    rep.append([float(row[i]), float(row[i+1]), float(row[i+2])])
                    i+=3                       
                data_syn.append(rep)
                labels_syn.append(1)

    with open (ID_csv, newline='') as file:
        reader = csv.reader(file, delimiter=',')
        for index, row in enumerate(reader):
            if row[0] in files_ID:
                rep = []
                i = 1
                while i < len(row[1:]):
                    rep.append([float(row[i]), float(row[i+1]), float(row[i+2])])
                    i+=3                       
                data_ID.append(rep)
                labels_ID.append(0)

    return np.array(data_syn), np.array(data_ID), np.array(labels_syn), np.array(labels_ID)

In [5]:
def normalize(data_1, data_2, data_3):
    data_1 = Normalizer().fit_transform(data_1)
    data_2 = Normalizer().fit_transform(data_2)
    data_3 = Normalizer().fit_transform(data_3)
    return data_1, data_2, data_3

In [6]:
def load_data(syn, GENERAL_DIR, data_dir): 

    method = "deepface"
    syn_csv = data_dir+"\\representations\{}-patients-{}.csv".format(syn, method)
    ID_csv  = data_dir+"\\representations\ID-controls-{}.csv".format(method)
    data_syn_df, data_ID_df, labels_syn_df, labels_ID_df = read_rep(syn, syn_csv, ID_csv, data_dir)
    
    method = "facereader-landmarks"
    syn_csv = GENERAL_DIR+ "\\features_facereader_landmarks_patient_groups.csv"
    ID_csv = GENERAL_DIR+ "\\features_facereader_landmarks_all_controls.csv" 
    data_syn_fr, data_ID_fr, _, _ = read_rep_landmarks(syn, syn_csv, ID_csv, data_dir)    
    
    method = "facereader-landmarks-distances"
    syn_csv = GENERAL_DIR+ "\\features_facereader_landmarks_distances_patient_groups_left_right.csv"
    ID_csv = GENERAL_DIR+ "\\features_facereader_landmarks_distances_all_controls_left_right.csv"    
    data_syn_dis, data_ID_dis, _,  _ = read_rep(syn, syn_csv, ID_csv, data_dir)  
    
    indices_to_keep = []
    for index, rep in enumerate(data_syn_dis):
        if not all(v == 0 for v in data_syn_dis[index]) and not all(v == 0 for v in data_ID_dis[index]):
            indices_to_keep.append(index)
                
    # all deepface data
    data_df = data_syn_df.tolist() + data_ID_df.tolist()
    labels_df = labels_syn_df.tolist() + labels_ID_df.tolist()
    
    # only deepface (that also has a facereader rep)
    data_syn_df_drop = data_syn_df[indices_to_keep]
    data_ID_df_drop = data_ID_df[indices_to_keep]
    data_df_drop = data_syn_df_drop.tolist() + data_ID_df_drop.tolist()
    
    # facereader landmarks 
    data_syn_fr = data_syn_fr[indices_to_keep]
    data_ID_fr = data_ID_fr[indices_to_keep]
    data_fr = data_syn_fr.tolist() + data_ID_fr.tolist()
    
    # only distance with facereader rep
    data_syn_dis = data_syn_dis[indices_to_keep]
    data_ID_dis = data_ID_dis[indices_to_keep]
    data_dis = data_syn_dis.tolist() + data_ID_dis.tolist()
    
    # labels with facereader rep
    labels_syn_df = labels_syn_df[indices_to_keep]
    labels_ID_df = labels_ID_df[indices_to_keep]
    labels = labels_syn_df.tolist() + labels_ID_df.tolist() 

    return np.array(data_df), np.array(data_df_drop), np.array(data_fr), np.array(data_dis), np.array(labels_df), np.array(labels)

In [7]:
def knn_classifier(data, labels):
    y_true, y_probs, y_preds = [], [], [] 
    loo = LeaveOneOut()

    # leave one out split and make prediction
    for train_index, test_index in loo.split(data):
        y_true.append(labels[test_index])
        
        X_train, X_test = np.array(data[train_index]), data[test_index]
        y_train, _ = np.array(labels[train_index]), labels[test_index]
        
        model = KNeighborsClassifier(n_neighbors=3, weights='distance')
        model.fit(X_train, y_train)

        y_probs.append(model.predict_proba(X_test.reshape(1, -1))[:,1])
        y_preds.append(model.predict(X_test.reshape(1, -1)))
        
        del model

    # based on all predictions make aroc curve and confusion matrix
    aroc = roc_auc_score(y_true, y_probs)
    tn, fp, fn, tp = confusion_matrix(y_true, y_preds).ravel()
    spec = tn / (tn+fp)  
    sens = tp / (tp+fn)
    f1 = f1_score(y_true, y_preds)
                
    return aroc, spec, sens, f1, y_true, y_probs, y_preds

In [8]:
def pointnet_classifier(data, labels):
    y_true, y_probs, y_preds = [], [], [] 
    loo = LeaveOneOut()
    
    for train_index, test_index in loo.split(data):
        y_true.append(labels[test_index])

        X_train, X_test = np.array(data[train_index]), data[test_index]
        y_train, _ = np.array(labels[train_index]), labels[test_index]

        model = pointnet_model.generate_model()
        model.fit(x=X_train, y=y_train, batch_size=BATCH_SIZE, epochs=4, shuffle=True)

        y_pred_array = model.predict(X_test)
        y_pred = tf.math.argmax(y_pred_array, -1).numpy()

        y_probs.append(y_pred_array[0][1])
        y_preds.append(y_pred) 
        
        del model
        
    aroc = roc_auc_score(y_true, y_probs)
    tn, fp, fn, tp = confusion_matrix(y_true, y_preds).ravel()
    spec = tn / (tn+fp)  
    sens = tp / (tp+fn)
    f1 = f1_score(y_true, y_preds)
    
    return aroc, spec, sens, f1, y_true, y_probs, y_preds

In [9]:
def randomforest_classifier(data, labels):
    y_true, y_probs, y_preds = [], [], [] 
    loo = LeaveOneOut()
    
    for train_index, test_index in loo.split(data):
        y_true.append(labels[test_index])

        X_train, X_test = np.array(data[train_index]), data[test_index]
        y_train, _ = np.array(labels[train_index]), labels[test_index]

        model = RandomForestClassifier(n_estimators=10)
        model.fit(X_train, y_train)

        y_probs.append(model.predict_proba(X_test.reshape(1, -1))[:,1])
        y_preds.append(model.predict(X_test.reshape(1, -1)))
        
        del model

    aroc = roc_auc_score(y_true, y_probs)
    tn, fp, fn, tp = confusion_matrix(y_true, y_preds).ravel()
    spec = tn / (tn+fp)  
    sens = tp / (tp+fn)
    f1 = f1_score(y_true, y_preds)
    
    return aroc, spec, sens, f1, y_true, y_probs, y_preds

In [11]:
# filter version
GENERAL_DIR = r"H:\Genetica Projecten\Facial Recognition\Studenten en Onderzoekers\Fien" 
BATCH_SIZE = 8
syn_list = ['ADNP', 'ANKRD11', 'CDK13', 'DEAF1', 'DYRK1A', 'EHMT1', 'FBXO11', 'KDVS', 'SON', 'WAC', 'YY1'] 

## open file 
results_file = open("results/ensemble_results_3_models_2.txt", "w")

# read in all data (per syndrome) which has a facereader and deepface representation
for syn in syn_list:
    
    data_dir = GENERAL_DIR + "\\{}".format(syn) 
    print("Syndrome that will be classified: {} \n\n".format(syn))
    results_file.write("Syndrome: {}\n".format(syn))
    
    data_df_all, data_df, data_fr, data_dis, labels_df_all, labels = load_data(syn, GENERAL_DIR, data_dir)    
      
    # all predictions 
    print("deepface", data_df.shape)   
    print("pointnet", data_fr.shape)  
    print("distances", data_dis.shape)
    
    data_df_all, data_df, data_dis = normalize(data_df_all, data_df, data_dis)   
    
    # DEEPFACE - KNN - all
    aroc_df_all, spec_df_all, sens_df_all, f1_df_all, _, _, _ = knn_classifier(data_df_all, labels_df_all)
    results_file.write("Deepface with {} patients and {} controls\n".format(labels_df_all.tolist().count(1), labels_df_all.tolist().count(0)))
    results_file.write("AROC: {:.4f}, spec: {:.4f}, sens: {:.4f}, F1: {:.4f}\n\n".format(aroc_df_all, spec_df_all, sens_df_all, f1_df_all))

    # DEEPFACE - KNN 
    aroc_df, spec_df, sens_df, f1_df, y_true_df, y_probs_df, y_preds_df = knn_classifier(data_df, labels)
    results_file.write("Deepface with {} patients and {} controls\n".format(labels.tolist().count(1), labels.tolist().count(0)))
    results_file.write("AROC: {:.4f}, spec: {:.4f}, sens: {:.4f}, F1: {:.4f}\n\n".format(aroc_df, spec_df, sens_df, f1_df))
           
    # POINTNET 
    aroc_pn, spec_pn, sens_pn, f1_pn, y_true_pn, y_probs_pn, y_preds_pn = pointnet_classifier(data_fr, labels)
    results_file.write("Pointnet with {} patients and {} controls\n".format(labels.tolist().count(1), labels.tolist().count(0)))
    results_file.write("AROC: {:.4f}, spec: {:.4f}, sens: {:.4f}, F1: {:.4f}\n\n".format(aroc_pn, spec_pn, sens_pn, f1_pn))
                       
    # RANDOM FOREST 
    aroc_rf, spec_rf, sens_rf, f1_rf, y_true_rf, y_probs_rf, y_preds_rf = randomforest_classifier(data_dis, labels)
    results_file.write("Random Forest with {} patients and {} controls\n".format(labels.tolist().count(1), labels.tolist().count(0)))
    results_file.write("AROC: {:.4f}, spec: {:.4f}, sens: {:.4f}, F1: {:.4f}\n\n".format(aroc_rf, spec_rf, sens_rf, f1_rf))
    
    #ensemble mean 
    ensemble_probs, ensemble_preds = [], []
    for index, _ in enumerate(y_true_df):
        mean_prob = np.mean([y_probs_df[index], y_probs_pn[index], y_probs_rf[index]])
        ensemble_probs.append(mean_prob)
        
        mode_pred = mode([y_preds_df[index][0], y_preds_pn[index][0], y_preds_rf[index][0]])
        ensemble_preds.append(mode_pred)
        
    aroc_ensemble = roc_auc_score(y_true_df, ensemble_probs)
    tn_en, fp_en, fn_en, tp_en = confusion_matrix(y_true_df, ensemble_preds).ravel()
    spec_ensemble = tn_en / (tn_en+fp_en)  
    sens_ensemble = tp_en / (tp_en+fn_en)
    f1_ensemble = f1_score(y_true_df, ensemble_preds)
    
    results_file.write("Ensemble classifier mean and mode \n".format(labels.tolist().count(1), labels.tolist().count(0)))
    results_file.write("AROC: {:.4f}, spec: {:.4f}, sens: {:.4f}, F1: {:.4f}\n\n".format(aroc_ensemble, spec_ensemble, sens_ensemble, f1_ensemble))
    
results_file.close()

Syndrome that will be classified: ADNP 


deepface (26, 4096)
pointnet (26, 510, 3)
distances (26, 72630)
Epoch 1/4
4/4 [==============================] - 1s 223ms/step - loss: 1.3148 - sparse_categorical_accuracy: 0.6000
Epoch 2/4
4/4 [==============================] - 1s 211ms/step - loss: 1.0170 - sparse_categorical_accuracy: 0.8000
Epoch 3/4
4/4 [==============================] - 1s 207ms/step - loss: 1.1798 - sparse_categorical_accuracy: 0.8000
Epoch 4/4
4/4 [==============================] - 1s 207ms/step - loss: 0.9979 - sparse_categorical_accuracy: 0.8000
Epoch 1/4
4/4 [==============================] - 1s 205ms/step - loss: 1.1782 - sparse_categorical_accuracy: 0.6800
Epoch 2/4
4/4 [==============================] - 1s 201ms/step - loss: 1.2561 - sparse_categorical_accuracy: 0.7200
Epoch 3/4
4/4 [==============================] - 1s 223ms/step - loss: 0.9921 - sparse_categorical_accuracy: 0.9200
Epoch 4/4
4/4 [==============================] - 1s 217ms/step - loss: 1.0931 - sp

4/4 [==============================] - 1s 201ms/step - loss: 0.9570 - sparse_categorical_accuracy: 0.8400
Epoch 3/4
4/4 [==============================] - 1s 211ms/step - loss: 1.1952 - sparse_categorical_accuracy: 0.8800
Epoch 4/4
4/4 [==============================] - 1s 206ms/step - loss: 0.9688 - sparse_categorical_accuracy: 0.8800
Epoch 1/4
4/4 [==============================] - 1s 200ms/step - loss: 1.0933 - sparse_categorical_accuracy: 0.6800
Epoch 2/4
4/4 [==============================] - 1s 212ms/step - loss: 1.2218 - sparse_categorical_accuracy: 0.8000
Epoch 3/4
4/4 [==============================] - 1s 196ms/step - loss: 0.9679 - sparse_categorical_accuracy: 0.9600
Epoch 4/4
4/4 [==============================] - 1s 189ms/step - loss: 1.2510 - sparse_categorical_accuracy: 0.8400
Epoch 1/4
4/4 [==============================] - 1s 231ms/step - loss: 1.1178 - sparse_categorical_accuracy: 0.7200
Epoch 2/4
4/4 [==============================] - 1s 221ms/step - loss: 1.0586 - sp

4/4 [==============================] - 1s 240ms/step - loss: 1.1435 - sparse_categorical_accuracy: 0.8387
Epoch 4/4
4/4 [==============================] - 1s 256ms/step - loss: 1.1681 - sparse_categorical_accuracy: 0.9032
Epoch 1/4
4/4 [==============================] - 1s 258ms/step - loss: 1.2840 - sparse_categorical_accuracy: 0.5806
Epoch 2/4
4/4 [==============================] - 1s 243ms/step - loss: 1.3748 - sparse_categorical_accuracy: 0.6774
Epoch 3/4
4/4 [==============================] - 1s 254ms/step - loss: 1.2675 - sparse_categorical_accuracy: 0.6774
Epoch 4/4
4/4 [==============================] - 1s 251ms/step - loss: 1.1712 - sparse_categorical_accuracy: 0.8065
Epoch 1/4
4/4 [==============================] - 1s 245ms/step - loss: 1.2763 - sparse_categorical_accuracy: 0.5484
Epoch 2/4
4/4 [==============================] - 1s 255ms/step - loss: 1.0504 - sparse_categorical_accuracy: 0.8065
Epoch 3/4
4/4 [==============================] - 1s 253ms/step - loss: 1.1144 - sp

4/4 [==============================] - 1s 261ms/step - loss: 0.9997 - sparse_categorical_accuracy: 0.7419
Epoch 2/4
4/4 [==============================] - 1s 252ms/step - loss: 1.0687 - sparse_categorical_accuracy: 0.8387
Epoch 3/4
4/4 [==============================] - 1s 256ms/step - loss: 1.1869 - sparse_categorical_accuracy: 0.8387
Epoch 4/4
4/4 [==============================] - 1s 256ms/step - loss: 1.0177 - sparse_categorical_accuracy: 0.8065
Epoch 1/4
4/4 [==============================] - 1s 264ms/step - loss: 1.2942 - sparse_categorical_accuracy: 0.5161
Epoch 2/4
4/4 [==============================] - 1s 256ms/step - loss: 1.2722 - sparse_categorical_accuracy: 0.7419
Epoch 3/4
4/4 [==============================] - 1s 256ms/step - loss: 1.4122 - sparse_categorical_accuracy: 0.7742
Epoch 4/4
4/4 [==============================] - 1s 252ms/step - loss: 1.1408 - sparse_categorical_accuracy: 0.8387
Epoch 1/4
4/4 [==============================] - 1s 260ms/step - loss: 1.2276 - sp

5/5 [==============================] - 1s 239ms/step - loss: 0.8401 - sparse_categorical_accuracy: 0.9429
Epoch 3/4
5/5 [==============================] - 1s 242ms/step - loss: 0.8779 - sparse_categorical_accuracy: 1.0000
Epoch 4/4
5/5 [==============================] - 1s 238ms/step - loss: 0.8509 - sparse_categorical_accuracy: 0.9714
Epoch 1/4
5/5 [==============================] - 1s 247ms/step - loss: 1.0374 - sparse_categorical_accuracy: 0.6857
Epoch 2/4
5/5 [==============================] - 1s 242ms/step - loss: 1.1662 - sparse_categorical_accuracy: 0.7429
Epoch 3/4
5/5 [==============================] - 1s 242ms/step - loss: 1.2305 - sparse_categorical_accuracy: 0.8000
Epoch 4/4
5/5 [==============================] - 1s 232ms/step - loss: 1.1950 - sparse_categorical_accuracy: 0.7714
Epoch 1/4
5/5 [==============================] - 1s 270ms/step - loss: 1.4548 - sparse_categorical_accuracy: 0.4857
Epoch 2/4
5/5 [==============================] - 1s 243ms/step - loss: 1.2247 - sp

...

13/13 [==============================] - 5s 400ms/step - loss: 1.7748 - sparse_categorical_accuracy: 0.4444
Epoch 3/4
13/13 [==============================] - 5s 400ms/step - loss: 1.6043 - sparse_categorical_accuracy: 0.4848
Epoch 4/4
13/13 [==============================] - 5s 405ms/step - loss: 1.2534 - sparse_categorical_accuracy: 0.5556
Epoch 1/4
13/13 [==============================] - 5s 395ms/step - loss: 1.5078 - sparse_categorical_accuracy: 0.5859
Epoch 2/4
13/13 [==============================] - 5s 407ms/step - loss: 1.6882 - sparse_categorical_accuracy: 0.4949
Epoch 3/4
13/13 [==============================] - 5s 395ms/step - loss: 1.4042 - sparse_categorical_accuracy: 0.5354
Epoch 4/4
13/13 [==============================] - 5s 398ms/step - loss: 1.4409 - sparse_categorical_accuracy: 0.4848
Epoch 1/4
13/13 [==============================] - 5s 393ms/step - loss: 1.6095 - sparse_categorical_accuracy: 0.5758
Epoch 2/4
13/13 [==============================] - 5s 388ms/step -

2/2 [==============================] - 0s 186ms/step - loss: 1.1867 - sparse_categorical_accuracy: 0.6667
Epoch 4/4
2/2 [==============================] - 0s 180ms/step - loss: 1.1987 - sparse_categorical_accuracy: 0.7333
Epoch 1/4
2/2 [==============================] - 0s 195ms/step - loss: 1.3580 - sparse_categorical_accuracy: 0.4000
Epoch 2/4
2/2 [==============================] - 0s 179ms/step - loss: 1.3846 - sparse_categorical_accuracy: 0.6000
Epoch 3/4
2/2 [==============================] - 0s 180ms/step - loss: 0.9722 - sparse_categorical_accuracy: 0.9333
Epoch 4/4
2/2 [==============================] - 0s 179ms/step - loss: 1.3438 - sparse_categorical_accuracy: 0.6667
Epoch 1/4
2/2 [==============================] - 0s 183ms/step - loss: 1.1429 - sparse_categorical_accuracy: 0.6000
Epoch 2/4
2/2 [==============================] - 0s 175ms/step - loss: 1.0554 - sparse_categorical_accuracy: 0.6667
Epoch 3/4
2/2 [==============================] - 0s 179ms/step - loss: 1.0289 - sp

3/3 [==============================] - 1s 184ms/step - loss: 1.0202 - sparse_categorical_accuracy: 1.0000
Epoch 1/4
3/3 [==============================] - 1s 192ms/step - loss: 0.9039 - sparse_categorical_accuracy: 0.7647
Epoch 2/4
3/3 [==============================] - 1s 184ms/step - loss: 1.0787 - sparse_categorical_accuracy: 0.7059
Epoch 3/4
3/3 [==============================] - 1s 183ms/step - loss: 0.8951 - sparse_categorical_accuracy: 0.9412
Epoch 4/4
3/3 [==============================] - 1s 179ms/step - loss: 0.8642 - sparse_categorical_accuracy: 0.8824
Epoch 1/4
3/3 [==============================] - 1s 181ms/step - loss: 1.4805 - sparse_categorical_accuracy: 0.5882
Epoch 2/4
3/3 [==============================] - 1s 178ms/step - loss: 1.2882 - sparse_categorical_accuracy: 0.7059
Epoch 3/4
3/3 [==============================] - 1s 183ms/step - loss: 1.3222 - sparse_categorical_accuracy: 0.8235
Epoch 4/4
3/3 [==============================] - 1s 178ms/step - loss: 1.0039 - sp

2/2 [==============================] - 0s 136ms/step - loss: 1.1141 - sparse_categorical_accuracy: 0.6154
Epoch 2/4
2/2 [==============================] - 0s 125ms/step - loss: 0.8745 - sparse_categorical_accuracy: 0.7692
Epoch 3/4
2/2 [==============================] - 0s 120ms/step - loss: 1.1932 - sparse_categorical_accuracy: 0.6154
Epoch 4/4
2/2 [==============================] - 0s 122ms/step - loss: 0.8732 - sparse_categorical_accuracy: 0.9231
Epoch 1/4
2/2 [==============================] - 0s 135ms/step - loss: 1.5664 - sparse_categorical_accuracy: 0.6154
Epoch 2/4
2/2 [==============================] - 0s 117ms/step - loss: 1.0350 - sparse_categorical_accuracy: 0.7692
Epoch 3/4
2/2 [==============================] - 0s 125ms/step - loss: 1.7482 - sparse_categorical_accuracy: 0.5385
Epoch 4/4
2/2 [==============================] - 0s 125ms/step - loss: 1.0335 - sparse_categorical_accuracy: 0.8462
Epoch 1/4
2/2 [==============================] - 0s 135ms/step - loss: 1.3206 - sp

ResourceExhaustedError:  OOM when allocating tensor with shape[8,510,512] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node gradient_tape/pointnet/batch_normalization_6933/moments/BroadcastTo_1 (defined at <ipython-input-8-df07ece83031>:12) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_3933420]

Function call stack:
train_function
